# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [62]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

In [63]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [64]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.columns = df.columns.str.replace('custateomer', 'customer').str.replace('custateomer_lifetime_value', 'customer_lifetime_value')
df.columns = df.columns.str.replace('vehicle_class', 'vehicule_class')
df.columns = df.columns.str.replace('employmentstatus', 'employment_status')

df.columns

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employment_status', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicule_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')

In [65]:
df.shape

(10910, 26)

In [66]:
df['unnamed:_0']

0            0
1            1
2            2
3            3
4            4
         ...  
10905    10905
10906    10906
10907    10907
10908    10908
10909    10909
Name: unnamed:_0, Length: 10910, dtype: int64

In [67]:
df = df.drop(columns=['unnamed:_0'])

In [68]:
df.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicule_class,vehicle_size,vehicle_type
0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,23675,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [95]:
#1-
filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]
filtered_df

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicule_class,vehicle_size,vehicle_type
189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,51643,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,71210,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,16181,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,0,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,FN44127,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,20978,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2/8/11,Employed,M,55687,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,NaN
10487,IX60941,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,20978,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,QO62792,Oregon,7840.165778,Yes,Extended,College,1/14/11,Employed,M,58414,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


In [99]:
filtered_df.shape

(67, 25)

In [103]:
#2-
filtered_df2 = df[df['response'] == 'Yes']
grouped_df = filtered_df2.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().reset_index()
grouped_df

,policy_type,gender,total_claim_amount
0,Corporate Auto,F,433.738499
1,Corporate Auto,M,408.582459
2,Personal Auto,F,452.965929
3,Personal Auto,M,457.010178
4,Special Auto,F,453.280164
5,Special Auto,M,429.527942


The average total_claim_amount does not vary drastically between the different policy types. The differences are relatively small, indicating that the policy type might not be a strong determinant of claim amounts.

Gender Differences:
Corporate Auto: Females have a slightly higher average claim amount (433.74) compared to males (408.58).
Personal Auto: Males have a marginally higher average claim amount (457.01) than females (452.97).
Special Auto: Females have a higher average claim amount (453.28) compared to males (429.53).

The small differences between genders and policy types suggest that insurance companies could refine their marketing and product offerings slightly differently for males and females, particularly in the Corporate Auto and Special Auto segments where the differences are more noticeable. The relatively small variation in average claim amounts might indicate that gender and policy type are not the most significant factors in determining claim amounts. 

In [118]:
#3-
state_counts = df['state'].value_counts().reset_index()
state_counts

,state,count
0,California,3552
1,Oregon,2909
2,Arizona,1937
3,Nevada,993
4,Washington,888


All the states listed have a significant number of customers, indicating a strong presence in these regions. 
California has the highest number of customers, followed by Oregon and Arizona. 

In [123]:
#4-
lifetime_value_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()
lifetime_value_stats

,education,gender,max,min,median
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5577.669457
6,High School or Below,F,55277.44589,2144.921535,6039.553187
7,High School or Below,M,83325.38119,1940.981221,6286.731006
8,Master,F,51016.06704,2417.777032,5729.855012
9,Master,M,50568.25912,2272.307310,5579.099207


The max, min, and median customer lifetime value varies across different education levels and genders. Generally, higher education levels such as Master’s and Doctor tend to have higher maximum and median values for customer_lifetime_value, indicating that more educated customers may have higher lifetime values.

For each education level, there are variations between genders:
Bachelor’s: Female customers have slightly lower median values compared to male customers.
Master’s: Female customers have a slightly lower median lifetime value than male customers, though the maximum values are similar.
Doctor's: Male customers tend to have higher maximum and median values compared to female customers.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [128]:
print(df['effective_to_date'].head())

0   2011-02-18
1   2011-01-18
2   2011-02-10
3   2011-01-11
4   2011-01-17
Name: effective_to_date, dtype: datetime64[ns]


In [130]:
#5-
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], format='%Y-%m-%d')
df['month'] = df['effective_to_date'].dt.to_period('M')

policy_counts = df.groupby(['state', 'month']).size().reset_index(name='number_of_policies')

pivot_table = policy_counts.pivot(index='state', columns='month', values='number_of_policies').fillna(0)
pivot_table

month,2011-01,2011-02
state,,
Arizona,1008,929
California,1918,1634
Nevada,551,442
Oregon,1565,1344
Washington,463,425


In [136]:
#6- 
state_totals = policy_counts.groupby('state')['number_of_policies'].sum().reset_index()

top_states = state_totals.sort_values(by='number_of_policies', ascending=False).head(3)
top_states_list = top_states['state'].tolist()

top_states_policy_counts = policy_counts[policy_counts['state'].isin(top_states_list)]

pivot_table_top_states = top_states_policy_counts.pivot(index='state', columns='month', values='number_of_policies').fillna(0)
pivot_table_top_states

month,2011-01,2011-02
state,,
Arizona,1008,929
California,1918,1634
Oregon,1565,1344


In [142]:
#7-
marketing_channels = ['sales_channel', 'renew_offer_type', 'coverage', 'policy_type']

melted_df = df.melt(id_vars=['response'], value_vars=marketing_channels, var_name='marketing_channel', value_name='channel_value')

melted_df = melted_df.dropna(subset=['channel_value'])

response_counts = melted_df.groupby(['marketing_channel', 'channel_value', 'response']).size().unstack(fill_value=0)

response_counts['total'] = response_counts['Yes'] + response_counts['No']
response_counts['response_rate'] = response_counts['Yes'] / response_counts['total']

response_rate_df = response_counts.reset_index()

response_rate_df[['marketing_channel', 'channel_value', 'response_rate']]

response,marketing_channel,channel_value,response_rate
0,coverage,Basic,0.143698
1,coverage,Extended,0.138989
2,coverage,Premium,0.147312
3,policy_type,Corporate Auto,0.146685
4,policy_type,Personal Auto,0.140397
5,policy_type,Special Auto,0.162228
6,renew_offer_type,Offer1,0.157396
7,renew_offer_type,Offer2,0.233881
8,renew_offer_type,Offer3,0.019729
9,renew_offer_type,Offer4,0.000000


Policy Type: Special Auto has the highest response rate (0.162) among policy types, indicating that customers may be more receptive to this type of policy.

Renew Offer Type: Offer2 has the highest response rate (0.234), significantly higher than other offers, suggesting that this offer is the most attractive to customers. Offer4 has a response rate of 0.000, meaning it is ineffective in generating positive responses and may need to be reevaluated or discontinued.

Sales Channel: Agent has the highest response rate (0.191), which indicates that sales through agents are the most effective at eliciting a positive response. Call Center and Web channels have lower response rates compared to the Agent channel, suggesting that these channels are less effective in converting prospects.

Coverage: the response rates are relatively close across different coverage types, with only a slight variation. Premiumcoverage is the highest response with 0,147.
